In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import sys
import itertools
import warnings
import random
from pandas.api.types import is_numeric_dtype
warnings.filterwarnings('ignore')

In [2]:
train_data_name=input("Enter the name of train data file [(ex) harris.dat] : ") # data name
test_data_name=input("Enter the name of test data file [(ex) harris.dat] : ") 
coding_fm=int(input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")) # data separator
separator_fm={coding_fm ==1 : " "}.get(True, ",")
res_pos=int(input("Enter the column position of the response variable : [from 1 to p] : "))
header=input("Does the data have column header? (y/n) : ")
if(header=="y") : trdata=pd.read_csv(train_data_name, sep=separator_fm) # loading data
else : trdata=pd.read_csv(train_data_name, sep=separator_fm, header=None) # loading data
if(header=="y") : tstdata=pd.read_csv(test_data_name, sep=separator_fm) # loading data
else : tstdata=pd.read_csv(test_data_name, sep=separator_fm, header=None) # loading data
out_name=input("Enter the output file name to export [(ex) result.txt] : ")

# Extracting X and y
data = pd.DataFrame(trdata)
X= data.drop([data.columns[res_pos-1]], axis=1 )
Y= data.iloc[:, res_pos-1 ].values

#Extracting test X and y
tstdata = pd.DataFrame(tstdata)
tstX= tstdata.drop([tstdata.columns[res_pos-1]], axis=1 )
tstY= tstdata.iloc[:, res_pos-1 ].values

Enter the name of train data file [(ex) harris.dat] : pid.dat
Enter the name of test data file [(ex) harris.dat] : pidtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Enter the column position of the response variable : [from 1 to p] : 8
Does the data have column header? (y/n) : n
Enter the output file name to export [(ex) result.txt] : result.txt


In [3]:
#change the y value of 1,2 to 0,1
def y_change(Y):
  changed_y = []
  for i in range(len(Y)):
    if Y[i] ==np.unique(Y)[0]:
      changed_y.append(0)
    elif Y[i] ==np.unique(Y)[1]:
      changed_y.append(1)
  return np.array(changed_y)

In [4]:
#first classifier
dc = DecisionTreeClassifier(max_depth=3)
CY = y_change(Y)
dc.fit(X,CY)

cm = []
for i in range(len(X)):
  cm.append(dc.predict_proba(X)[i][1])

In [5]:
# start B=101 regression
lr = 0.1 #learning rate
cut_off = 0.5 #cutoff
B=101 # n_ensemble

trees = {}
for i in range(B):
  trees[f'h{i+1}'] = DecisionTreeRegressor(max_depth=3)
  res = CY -cm
  trees[f'h{i+1}'].fit(X,res)
  h = trees[f'h{i+1}'].predict(X)
  cm = cm + lr * h


In [6]:
#result

result = []
for i in range(len(tstX)):
  result.append(dc.predict_proba(tstX)[i][1])

for i in range(B):
  result = result + lr * trees[f'h{i+1}'].predict(tstX)

result[result>=0.5] = 1
result[result<0.5] =0

sys.stdout = open(out_name,'w')

print('    ')
print( 'Confusion Matrix( Gradient Boosting)')
print('----------------------------------------------------')
confusion_tst = confusion_matrix(y_change(tstY), result)

accu_tst = 0
for i in range(len(np.unique(y_change(tstY)))):
  accu_tst = accu_tst + confusion_tst[i][i]
accuracy_tst = accu_tst / len(tstdata)

print('           predicted class \n Actual 1 ' ,confusion_tst[0], '\n class  2 ', confusion_tst[1])
for i in range(2, len(np.unique(y_change(tstY)))) :
  print(f'        {i+1} ', confusion_tst[i])
print('model summary')
print('----------------------------------')
print('Overall accuracy = ' ,accuracy_tst)